In [ ]:
# WARNING: advised to install a specific version, e.g. qrules==0.1.2
%pip install -q qrules[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Visualize solutions

````{margin}
```{warning}
{class}`graphviz.Source` requires your system to have DOT installed, see {doc}`Installation <graphviz:index>`.
```
````

The {mod}`~qrules.io` module allows you to convert {class}`.MutableTransition`, {class}`.Topology` instances, and {class}`.ProblemSet`s to [DOT language](https://graphviz.org/doc/info/lang.html) with {func}`.asdot`. You can visualize its output with third-party libraries, such as [Graphviz](https://graphviz.org). This is particularly useful after running {meth}`~.StateTransitionManager.find_solutions`, which produces a {class}`.ReactionInfo` object with a {class}`.list` of {class}`.MutableTransition` instances (see {doc}`/usage/reaction`).

## Topologies

First of all, here are is an example of how to visualize a group of {class}`.Topology` instances. We use {func}`.create_isobar_topologies` and {func}`.create_n_body_topology` to create a few standard topologies.

In [ ]:
import graphviz
from IPython.display import display

import qrules
from qrules.particle import Spin
from qrules.topology import create_isobar_topologies, create_n_body_topology
from qrules.transition import State

In [ ]:
topology = create_n_body_topology(2, 4)
graphviz.Source(qrules.io.asdot(topology, render_initial_state_id=True))

Note the IDs of the {attr}`~.Topology.nodes` is also rendered if there is more than node:

In [ ]:
topologies = create_isobar_topologies(4)
graphviz.Source(qrules.io.asdot(topologies))

This can be turned on or off with the arguments of {func}`.asdot`:

In [ ]:
topologies = create_isobar_topologies(3)
graphviz.Source(qrules.io.asdot(topologies, render_node=False))

{func}`.asdot` provides other options as well:

In [ ]:
topologies = create_isobar_topologies(5)
dot = qrules.io.asdot(
    topologies[0],
    render_final_state_id=False,
    render_resonance_id=True,
    render_node=False,
)
display(graphviz.Source(dot))

(problem-sets)=
## {class}`.ProblemSet`s

As noted in {doc}`reaction`, the {class}`.StateTransitionManager` provides more control than the façade function {func}`.generate_transitions`. One advantages, is that the {class}`.StateTransitionManager` first generates a set of {class}`.ProblemSet`s with {meth}`.create_problem_sets` that you can further configure if you wish.

In [ ]:
from qrules.settings import InteractionType

stm = qrules.StateTransitionManager(
    initial_state=["J/psi(1S)"],
    final_state=["K0", "Sigma+", "p~"],
    formalism="canonical-helicity",
)
stm.set_allowed_interaction_types([InteractionType.STRONG, InteractionType.EM])
problem_sets = stm.create_problem_sets()

Note that the output of {meth}`.create_problem_sets` is a {obj}`dict` with {obj}`float` values as keys (representing the interaction strength) and {obj}`list`s of {obj}`.ProblemSet`s as values.

In [ ]:
sorted(problem_sets, reverse=True)

In [ ]:
problem_set = problem_sets[60.0][0]
dot = qrules.io.asdot(problem_set, render_node=True)
graphviz.Source(dot)

## Quantum number solutions

As noted in {ref}`usage/reaction:3. Find solutions`, a {obj}`.ProblemSet` can be fed to {meth}`.StateTransitionManager.find_solutions` directly to get a {obj}`.ReactionInfo` object. {obj}`.ReactionInfo` is a final result that consists of {obj}`.Particle`s, but in the intermediate steps, QRules works with sets of quantum numbers. One can inspect these intermediate generated quantum numbers by using {meth}`.find_quantum_number_transitions` and inspecting is output. Note that the resulting object is again a {obj}`dict` with strengths as keys and a list of solution as values.

In [ ]:
qn_solutions = stm.find_quantum_number_transitions(problem_sets)
{strength: len(values) for strength, values in qn_solutions.items()}

The list of solutions consist of a {obj}`tuple` of a {obj}`.QNProblemSet` (compare {ref}`problem-sets`) and a {obj}`.QNResult`:

In [ ]:
strong_qn_solutions = qn_solutions[3600.0]
qn_problem_set, qn_result = strong_qn_solutions[0]

In [ ]:
dot = qrules.io.asdot(qn_problem_set, render_node=True)
graphviz.Source(dot)

In [ ]:
dot = qrules.io.asdot(qn_result, render_node=True)
graphviz.Source(dot)

## {obj}`.StateTransition`s

After finding the {ref}`usage/visualize:Quantum number solutions`, QRules finds {obj}`.Particle` definitions that match these quantum numbers. All these steps are hidden in the convenience functions {meth}`.StateTransitionManager.find_solutions` and {func}`.generate_transitions`. In the following, we'll visualize the allowed transitions for the decay $\psi' \to \gamma\eta\eta$ as an example.

In [ ]:
import qrules

reaction = qrules.generate_transitions(
    initial_state="psi(2S)",
    final_state=["gamma", "eta", "eta"],
    allowed_interaction_types="EM",
)

As noted in {ref}`usage/reaction:3. Find solutions`, the {attr}`~.ReactionInfo.transitions` contain all spin projection combinations (which is necessary for the {mod}`ampform` package). It is possible to convert all these solutions to DOT language with {func}`~.asdot`. To avoid visualizing all solutions, we just take a subset of the {attr}`~.ReactionInfo.transitions`:

In [ ]:
dot = qrules.io.asdot(reaction.transitions[::50][:3])  # just some selection

This {class}`str` of [DOT language](https://graphviz.org/doc/info/lang.html) for the list of {class}`.MutableTransition` instances can then be visualized with a third-party library, for instance, with {class}`graphviz.Source`:

In [ ]:
import graphviz

dot = qrules.io.asdot(
    reaction.transitions[::50][:3], render_node=False
)  # just some selection
graphviz.Source(dot)

You can also serialize the DOT string to file with {func}`.io.write`. The file extension for a DOT file is `.gv`:

In [ ]:
qrules.io.write(reaction, "decay_topologies_with_spin.gv")

### Collapse graphs

Since this list of all possible spin projections {attr}`~.ReactionInfo.transitions` is rather long, it is often useful to use `strip_spin=True` or `collapse_graphs=True` to bundle comparable graphs. First, {code}`strip_spin=True` allows one collapse (ignore) the spin projections (we again show a selection only):

In [ ]:
dot = qrules.io.asdot(reaction.transitions[:3], strip_spin=True)
graphviz.Source(dot)

or, with stripped node properties:

In [ ]:
dot = qrules.io.asdot(reaction.transitions[:3], strip_spin=True, render_node=True)
graphviz.Source(dot)

```{note}
By default, {func}`.asdot` renders edge IDs, because they represent the (final) state IDs as well. In the example above, we switched this off.
```

If that list is still too much, there is {code}`collapse_graphs=True`, which bundles all graphs with the same final state groupings:

In [ ]:
dot = qrules.io.asdot(reaction, collapse_graphs=True, render_node=False)
graphviz.Source(dot)

### Other state renderings

The {meth}`~.FrozenTransition.convert` method makes it possible to convert the types of its {attr}`~.FrozenTransition.states`. This for instance allows us to only render the spin states on in a {class}`.Transition`:

::::{margin}

:::{tip}

We use the fact that a {obj}`.StateTransition` is frozen (and therefore hashable) to remove any duplicate transitions.

:::

::::

In [ ]:
spin_transitions = sorted({
    t.convert(lambda s: Spin(s.particle.spin, s.spin_projection))
    for t in reaction.transitions
})
some_selection = spin_transitions[::67][:3]
dot = qrules.io.asdot(some_selection, render_node=True)
graphviz.Source(dot)

Or any other properties of a {class}`.State`:

In [ ]:
def render_mass(state: State, digits: int = 3) -> str:
    mass = round(state.particle.mass, digits)
    width = round(state.particle.width, digits)
    if width == 0:
        return str(mass)
    return f"{mass}±{width}"


mass_transitions = sorted({
    t.convert(
        state_converter=render_mass,
        interaction_converter=lambda _: None,
    )
    for t in reaction.transitions
})
dot = qrules.io.asdot(mass_transitions[::10])
graphviz.Source(dot)

## Styling

<!-- cspell:ignore darkgreen fontcolor fontsize penwidth -->
The {func}`.asdot` function also takes [Graphviz attributes](https://graphviz.org/doc/info/attrs.html). These can be used to modify the layout of the whole figure. Examples are the [`size`](https://graphviz.org/docs/attrs/size), [`color`](https://graphviz.org/docs/attrs/color), and [`fontcolor`](https://graphviz.org/docs/attrs/fontcolor). Edges and nodes can be styled with `edge_style` and `node_style` respectively:

In [ ]:
dot = qrules.io.asdot(
    reaction.transitions[0],
    render_node=True,
    size=12,
    bgcolor="white",
    edge_style={
        "color": "red",
        "arrowhead": "open",
        "fontcolor": "blue",
        "fontsize": 25,
    },
    node_style={
        "color": "gray",
        "penwidth": 2,
        "shape": "ellipse",
        "style": "dashed",
    },
)
display(graphviz.Source(dot))